In [1]:
import pandas as pd
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
from lib.residental_data_source import ResidentialDataSource

In [2]:
data_source = ResidentialDataSource('src/clean_data.csv')
data_set = data_source.fetch_data_set()

## Аномалии и выбросы

In [3]:

data_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1607 entries, 0 to 1606
Columns: 123 entries, Unnamed: 0.1 to Количество проданных нежилых помещений
dtypes: float64(29), int64(40), object(54)
memory usage: 1.5+ MB


In [4]:
missing_values = data_set.isnull().sum()
missing_values

Unnamed: 0.1                                 0
Unnamed: 0                                   0
index()                                      0
Регион                                       0
Населенный пункт                             0
                                          ... 
Количество проданных квартир               844
Общая площадь проданных квартир            844
Процент распроданности                     844
Кумулятивная средняя стоимость кв.метра    844
Количество проданных нежилых помещений     844
Length: 123, dtype: int64

In [5]:
data_set['Продано квартир, м2'].describe().apply(lambda x: f"{x: .3f}")

count     1331.000
mean       263.380
std        357.527
min          0.000
25%         35.300
50%        150.430
75%        365.460
max       4380.470
Name: Продано квартир, м2, dtype: object

In [6]:
data_set['Продано квартир, руб'].describe().apply(lambda x: f"{x: .3f}")

count          1331.000
mean       35095359.731
std        45139228.627
min               0.000
25%         4440462.000
50%        21380600.000
75%        49662140.000
max       428464100.000
Name: Продано квартир, руб, dtype: object

<span style="color: red;">Максимальные значения выбиваются на порядок, похоже на аномалию!! </span>

## Пороги для отнесения значения к выбросам

In [7]:
def calculate_outliers(data):

    q25 = data.quantile(0.25)
    q75 = data.quantile(0.75)
    iqr = q75 - q25
    lower_bound = q25 - 1.5 * iqr
    upper_bound = q75 + 1.5 * iqr

    print(f"Нижняя граница: {lower_bound}")
    print(f"Верхняя граница: {upper_bound}")

    outliers = (data < lower_bound) | (data > upper_bound)
    print(f"Количество выбросов: {outliers.sum()}")
    
    # print(data_set[outliers])
    
    boundaries = (q25 - 1.5 * iqr, q75 + 1.5 * iqr)

    return boundaries

In [8]:
boundaries = calculate_outliers(data_set['Продано квартир, м2'])

Нижняя граница: -459.93999999999994
Верхняя граница: 860.6999999999999
Количество выбросов: 82


In [18]:
# замена выбросов на верхнюю границу


def correcting_outlier(df, column, upper_bound):
    df.loc[df[column] > upper_bound, column] = upper_bound
    return df


In [10]:
data = data_set.copy()
data = correcting_outlier(data, 'Продано квартир, м2', boundaries[1] )

In [13]:
boundaries_rub = calculate_outliers(data_set['Продано квартир, руб'])

Нижняя граница: -63392055.0
Верхняя граница: 117494657.0
Количество выбросов: 74


In [14]:
data = correcting_outlier(data, 'Продано квартир, руб', boundaries_rub[1] )

## Проверка исправленных выбросов

In [17]:
calculate_outliers(data['Продано квартир, м2'])

Нижняя граница: -459.93999999999994
Верхняя граница: 860.6999999999999
Количество выбросов: 0


(np.float64(-459.93999999999994), np.float64(860.6999999999999))

In [16]:
calculate_outliers(data['Продано квартир, руб'])

Нижняя граница: -63392055.0
Верхняя граница: 117494657.0
Количество выбросов: 0


(np.float64(-63392055.0), np.float64(117494657.0))